Numerical analysis (or data analysis) is what Python is especially good at.  And it's possibly why you want to learn Python?  Now we will use a simple project to guide you.  Let's use Python to play music.

In [ ]:
# This cell sets up helpers for demonstration in the course.
# Before the end of this session, you shouldn't take this as examples.

%matplotlib inline

# Import necessary modules.

import os

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Audio

# Define constants.

sampling_freq = 44100 # 44.1 kHz.

# Define demonstrative helper functions.

def time_data(duration, rate=sampling_freq):
    return np.linspace(0, duration, num=int(rate*duration))

def sine_data(freq, time, damp=0):
    data = 2**13 * np.sin(2*np.pi * freq * time)
    if damp: # Remove gitters.
        todamp = int(damp * sampling_freq)
        darr = np.arange(todamp)/(todamp-1)
        darr = np.sin(darr * np.pi / 2)
        data[:todamp] *= darr
        data[-todamp:] *= 1-darr
    return data

def note_freq(note):
    """Parse note symbol "a4", "c4", etc. and calculate frequency (in Hz)."""
    if ' ' == note:
        return 0.0
    f0 = 440.0 # a4
    note = note.lower()
    diff = {'a': 0, 'b': 2, 'c': -9, 'd': -7, 'e': -5, 'f': -4, 'g': -2}[note[0]]
    diff += {'#': 1, 'b': -1}.get(note[1], 0)
    diff += 12 * (int(note[-1])-4)
    return f0 * 2 ** (float(diff)/12)

def play_notes(duration, *notes):
    time = time_data(duration, sampling_freq)
    alist = [sine_data(note_freq(n), time) for n in notes]
    data = np.vstack(alist).sum(axis=0)
    display(Audio(data.astype('int16'), rate=sampling_freq))
    return time, data

def play_numbered(numbered_notation, bpm=90, base_octave=4):
    # Middle C is designated C4
    note_map = {'0':' ', '1':'c', '2':'d', '3':'e', '4':'f', '5':'g', '6':'a', '7':'b'}
    opt_symbols = (
        # octaves
        '^', 'v',
        # accidentals
        '#', 'b',
        # note length
        '-', '.', '_',
        # misc
        '|', ' ', ','
    )
    beat_sec = 60.0 / bpm
    signals = []
    # init numbered
    default_num = lambda: (base_octave, '', 1.0)
    octave, accidental, nbeat = default_num()
    for it, num in enumerate(numbered_notation):
        # raise/lower octave
        if num == '^':
            octave += 1
        elif num == 'v':
            octave -= 1
        
        # accidentals
        if num in '#b':
            accidental = num
        
        if num in opt_symbols:
            continue
            
        for ahead in numbered_notation[it+1:]:
            if '-' == ahead:
                nbeat += 1
            elif '.' == ahead:
                nbeat += 0.5
            elif '_' == ahead:
                nbeat /= 2
            else:
                break
        
        time = time_data(beat_sec * nbeat, sampling_freq)
        note = note_map[num]
        if note != ' ':
            note = note + accidental + str(octave)
        freq = note_freq(note)
        signal = sine_data(freq, time, damp=beat_sec*0.1)
        signals.append(signal)
        
        # reset to default
        octave, accidental, nbeat = default_num()
        
    data = np.hstack(signals)
    display(Audio(data.astype('int16'), rate=sampling_freq))

Run the following cells.  They'll show you what the above code do.

In [ ]:
print("Play the C chord:")
time_point, data = play_notes(3, 'c4', 'e4', 'g4')

print("Plot the first 1000 time points (around %.0f ms):" % (1.e6/sampling_freq))
_ = plt.plot(time_point[0:1000], data[0:1000])

In [ ]:
print("Twinkle Twinkle Little Star:")
play_numbered('1155|665-|4433|221-|5544|332-|5544|332-|1155|665-|4433|221-')

print("Hänschen Klein:")
play_numbered('533-|422-|1234|555-|533-|422-|1355|3--0'
              '2222|234-|3333|345-|533-|422-|1355|1--0')

print("Hänschen Klein at 120 BPM:")
play_numbered('533-|422-|1234|555-|533-|422-|1355|3--0'
              '2222|234-|3333|345-|533-|422-|1355|1--0', bpm=120)

print("Marshmello - TELL ME as 142 BPM:")
play_numbered('#5#1-#5|4-#1_#2_4|#5#1-^#1|^1-#5_#6_4' * 2, bpm=142)

Now, step by step, we'll show you how we made Python sing.

# Variables

Variables are the things that Python uses to keep track of information.  They are the handle to access the associated data.  Let's see an example:

In [ ]:
time = time_data(10)
plt.plot(time)

We created some data by calling the helper function `time_data()`, and named it to `time` the variable.

# Tables and Arrays

# Data Types

# Containers